In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# CSV 파일 불러오기
train_df = pd.read_csv('/content/drive/MyDrive/캡스톤 부동산/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/캡스톤 부동산/test.csv')
submission_df = pd.read_csv('/content/drive/MyDrive/캡스톤 부동산/submission.csv')

# 필요없는 열 제거
train_df_cleaned = train_df.drop(columns=['transaction_id', 'apartment_id', 'jibun', 'apt', 'addr_kr', 'transaction_date'])
test_df_cleaned = test_df.drop(columns=['transaction_id', 'apartment_id', 'jibun', 'apt', 'addr_kr', 'transaction_date'])

# train과 test 데이터를 합쳐서 범주형 데이터 인코딩 준비
combined_df = pd.concat([train_df_cleaned[['city', 'dong']], test_df_cleaned[['city', 'dong']]])

# 모든 데이터를 문자열로 변환 (범주형 데이터 인코딩 오류 방지)
combined_df = combined_df.astype(str)

# 범주형 데이터 인코딩 (city, dong)
for column in ['city', 'dong']:
    le = LabelEncoder()
    combined_df[column] = le.fit_transform(combined_df[column])
    # 인코딩 결과를 원래 데이터프레임으로 복사
    train_df_cleaned[column] = combined_df[column].iloc[:len(train_df_cleaned)]
    test_df_cleaned[column] = combined_df[column].iloc[len(train_df_cleaned):]

# 특성과 타겟 변수 분리
X = train_df_cleaned.drop(columns=['transaction_real_price'])
y = train_df_cleaned['transaction_real_price']

# 학습용 데이터와 테스트용 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 모델 학습
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# test 데이터 예측
test_predictions = model.predict(test_df_cleaned)

# 예측 결과를 submission 데이터프레임에 반영
submission_df['transaction_real_price'] = test_predictions

# 결과를 CSV 파일로 저장
submission_df.to_csv('/content/drive/MyDrive/캡스톤 부동산/submission_result.csv', index=False)

# 결과 확인
print(submission_df.head())


   transaction_id  transaction_real_price
0         1145756                31788.50
1         1198704                15894.60
2         1222384                12744.50
3         1179897                96601.50
4         1223091                54494.94
